In [1]:
#'''
#Todo: Add more lines in the relation C:\Users\win7\Documents\Fonte_data\sourcePort_to_shipperId.csv
#then test the program.
#Need to write a summary for smelted quantity of each material.
#
#'''


In [2]:
# Import
import pandas as pd
import numpy as np
import os
from datetime import date

In [3]:
# Read starting inventory file


### Open transaction data
The transaction file is obtained in Horne web page/ Applications/ Rapports Horne/ MAIT/ SIGI/ Transactions de materiel.  
Take the transactions from yesterday to today.  
Transactions sorted by time because some ports are both a destination and source, and we want to track their composition.  
To self: possibly use all transactions starting from a certain date. Accounts for transaction changes.

In [4]:
# Read transaction file.
dT = pd.read_csv(r'C:\Users\win7\Documents\Fonte_data\Transactions de matériel 1er dec.csv')
# Drop useless columns
dT = dT.loc[:,['TAS_ID', 'REVERSE_TAS_ID1', 'Textbox58', 'SHIPPER_CODE','ACTION_TIME']]
# Rename columns
dT.columns = ['sourcePort', 'destPort', 'DMT', 'shipperCode','time']
# Sort by time
dT = dT.sort_values(by=['time'])

dT.head(15)

,sourcePort,destPort,DMT,shipperCode,time
85,600,3008,"1,11",600,531
87,600,3008,"10,57",600,531
90,600,730,"0,09",600,531
86,600,3008,"7,33",600,531
100,600,798,"0,59",600,634
97,600,798,"18,63",600,634
125,841,798,"12,07",841,936
88,600,3008,"18,71",600,1105
89,600,3008,"18,56",600,1205
92,600,730,"0,07",600,1205


In [5]:
# Convert comma to decimal
dT.replace(',','.', regex=True, inplace=True)
# Convert to float
dT.loc[:,'DMT'] = pd.to_numeric(dT.loc[:,'DMT'])
dT.head()

,sourcePort,destPort,DMT,shipperCode,time
85,600,3008,1.11,600,531
87,600,3008,10.57,600,531
90,600,730,0.09,600,531
86,600,3008,7.33,600,531
100,600,798,0.59,600,634


In [6]:
# Keep rows if destination is to "Eglise"
# Todo eglisePorts read from csv
eglisePorts = [564,565,900,903,906,913,915,916,917,918,919,920,921,922,923,924]

dT = dT[dT['destPort'].isin(eglisePorts)]
dT.head()

,sourcePort,destPort,DMT,shipperCode,time
64,322,564,398.77,,62350
67,373,564,71.82,,62357
68,376,564,21.93,,62404
76,500,564,487.51,,62412
80,564,916,787.16,,62419


We want a unique identifier for each material, therefore we wish to convert the port numbers to an alphabetical shipper code.
shipperCode column currently contains both string shipper codes ('EAGL', 'STAL', etc.) and port numbers (812, 376, etc.). 

In [7]:
# shipperCode column contains port numbers and shipper codes
# Replace empty strings with NaN
#dT['shipperCode'] = dT['shipperCode'].replace(r'^\s*$', np.nan, regex=True)
#dT.head()

Now we load a dictionnary to assign a shipper code to each line of dT that has a port number value in the column 'shipperCode'.  
However, some port numbers are actually a mix of materials (ex: 304 is a mix of green concentrates, 385 is a mix of concentrates with nickel).  
If a port is composed of a single type of material, we treat it as a single material even if it is a mix of different materials (ex: all 304 is MTA despite having MTA, XKM, BCM, COPA, FLI).  
If not, we consider its composition (ex: 564 can be a mix of concentrates, pyrite and electronics).


In [8]:
# Load the dictionnary to replace numerical port numbers with shipper codes. shipper_code = f(source_port)
# This dictionnary must be up to date, as port numbers and materials may change.

shipID = pd.read_csv(r'C:\Users\win7\Documents\Fonte_data\sourcePort_to_shipperId.csv')
shipID.head()


,ShipperID,SourcePort
0,BCM,NaN
1,Pyrite_don_rouyn_gallen,498.0
2,NMP,520.0
3,NMP,521.0
4,XKM,NaN


In [58]:
# Remove row if SourcePort value is missing
shipID = shipID.loc[shipID['SourcePort'].notnull(),:]
shipID.shape
shipID

,ShipperID,SourcePort
1,Pyrite_don_rouyn_gallen,498.0
2,NMP,520.0
3,NMP,521.0
6,MTA,304.0
7,MTA,322.0
...,...,...
83,TMO,500.0
84,WOR,332.0
85,Smelter Grade,3008.0
86,Fine Carbone,777.0


In [10]:
# Create dictionary
shipID_dict = shipID.set_index('SourcePort')['ShipperID'].to_dict()
# This also works
# shipID_dict = dict(zip(shipID['SourcePort'], shipID['ShipperID']))

In [11]:
# Map shipperID to transactions
dT['shipperID'] = dT['sourcePort'].map(shipID_dict)
# Drop null values of shipperID
#dT = dT[dT['shipperID'].notnull()]
dT.head()
before = dT.copy()

In [12]:
before

,sourcePort,destPort,DMT,shipperCode,time,shipperID
64,322,564,398.77,,62350,MTA
67,373,564,71.82,,62357,Electronique Rx
68,376,564,21.93,,62404,LSN_et_INC4
76,500,564,487.51,,62412,TMO
80,564,916,787.16,,62419,Melange fil
...,...,...,...,...,...,...
112,804,921,16.00,804,140717,Internes
231,804,921,15.92,804,150904,Internes
118,812,922,15.22,812,152052,Smelter Grade
117,812,922,15.05,812,154008,Smelter Grade


Copy and overwrite alphabetical values from shipperCode to shipperID. shipperCode data is more reliable because it comes directly from the transactions data, whereas shipperID uses a manually updated dictionnary.


In [13]:
dT.loc[(~dT['shipperCode'].str.isnumeric()) & (~dT['shipperCode'].str.contains(r'^\s*$')), 'shipperID']\
= dT.loc[(~dT['shipperCode'].str.isnumeric()) & (~dT['shipperCode'].str.contains(r'^\s*$')), 'shipperCode']

In [14]:
# Show rows where an update was made
a = before[dT['shipperID'] != before['shipperID']]['shipperID']
b = dT[dT['shipperID'] != before['shipperID']]['shipperID']
comparison = pd.concat([a,b],axis=1)
comparison.set_axis(['before_shipperID','after_shipperID'], axis=1, inplace=True)
comparison.head()

,before_shipperID,after_shipperID
36,NaN,DUM
176,NaN,EAGL
178,NaN,EAGL
177,NaN,EAGL
38,NaN,SAB2


In [15]:
# if shipperID is null, and if shipperCode is notnull, copy shipperCode to shipperID
# Copies over the shipperCode given by the transaction data.
# This also copies over numerical ports to shipperID, which is not wanted.
dT['shipperID'] = np.where(dT['shipperCode'].notnull() & dT['shipperID'].isnull(),dT['shipperCode'], dT['shipperID'])
dT.head()

,sourcePort,destPort,DMT,shipperCode,time,shipperID
64,322,564,398.77,,62350,MTA
67,373,564,71.82,,62357,Electronique Rx
68,376,564,21.93,,62404,LSN_et_INC4
76,500,564,487.51,,62412,TMO
80,564,916,787.16,,62419,Melange fil


In [16]:
# Check if there are numeric values in shipperID. There should be none. If there are, review the sourcePort_to_shipperID file.
dT[dT['shipperID'].str.isnumeric()]

,sourcePort,destPort,DMT,shipperCode,time,shipperID


In [17]:
print(dT.to_string())

     sourcePort  destPort     DMT shipperCode    time        shipperID
64          322       564  398.77               62350              MTA
67          373       564   71.82               62357  Electronique Rx
68          376       564   21.93               62404      LSN_et_INC4
76          500       564  487.51               62412              TMO
80          564       916  787.16               62419      Melange fil
82          564       917  198.35               62427      Melange fil
66          354       917   55.73               62434             CAPE
83          566       918  108.17               62441              CUZ
84          589       918  103.25               62447             STAL
63          312       919    5.64               62453         Internes
71          438       919  286.57               62459              CLH
72          487       919   73.01               62516             BOLA
79          534       919   32.56               62521             BOLA
70    

In [ ]:
# Check if shipperCode value is a letter string (not numeric)
# Todo: Display rows containing alpha shipper codes.
#dT.loc[dT.loc[:,'shipperCode'].notnull(),'shipperCode'].str.isnumeric()
#non_numeric_rows = ~dT.loc[dT.loc[:,'shipperCode'].notnull(),'shipperCode'].str.isnumeric()

In [ ]:
#temp2 = dT.loc[dT.loc[:,'shipperCode'].notnull(),'shipperCode'].str.isnumeric()
#temp2
# find a way to select rows that are notnull and not numeric...

In [ ]:
#dT.loc[dT.loc[:,'shipperCode'].notnull(),'shipperCode'].str.isnumeric()

In [ ]:
#dT.loc[dT.loc[:,'shipperCode'].notnull(),'shipperCode']

In [ ]:
#dT[non_numeric_rows]

In [ ]:
# Find rows containing non-numeric and not-null
#dT.loc[non_numeric_rows,:]

In [ ]:
# ShipperCode column contains port numbers and shipper codes. 
# If it is a letter string, copy this to the shipperID column
# is not numeric how to find
#dT[dT.loc[:,'shipperCode'].notnull().isnumeric()

Read inventory.   
This is the starting inventory at Eglise, on the Monday of the week. Data must be entered manually. Names on the first column must exactly match the shipperID nomenclature, otherwise transactions will be omitted.

In [18]:
# Read prior inventory
inv = pd.read_csv(r'C:\Users\win7\Documents\Fonte_data\inv2.csv', index_col=0)
inv = inv.fillna(0)

inv

,564,560,900,903,906,913,915,916,917,918,919,920,921,922,923,924,925
XKM,367.0,0.0,264.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
FLI,0.0,0.0,604.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
MTA,0.0,0.0,954.0,0.0,0.0,0.0,0.0,66.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Pyrite_don_rouyn_gallen,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
BCM,366.0,0.0,178.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
DUM,0.0,0.0,0.0,6.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
NVA,110.0,0.0,0.0,44.0,0.0,0.0,0.0,10.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
LSN_et_INC4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
TIZ,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
FAS,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


The order of transaction matters, from oldest to newest.

In [19]:
# Add transactions to inventory
# This is the naive method, find a more elegant way later.
for k in range(dT.shape[0]):
    for j in inv.columns:
        #If the sourcePort is 564, and the destPort matches with inv column, consider the composition of 564
        if dT.iloc[k,0]==564 and str(dT.iloc[k,1])==j:
            # Remaining mass fraction in 564
            m = 1 - dT.loc[dT.index[k], 'DMT'] / inv.loc[:,'564'].sum()
            # Add mass to destPort
            inv.loc[:,j] += (1-m)*inv.loc[:,'564']
            print('Transferred {:.2f} tons from 564 to {}.'.format((1-m)*inv.loc[:,'564'].sum(),j))
            # Remove mass from 564
            inv.loc[:,'564'] = m * inv.loc[:,'564']
            
        else:
            for i in inv.index:
                # If shipperID and port number matches
                if dT.iloc[k,-1] == i and str(dT.iloc[k,1])==j:
                    inv.loc[i,j] += float(dT.iloc[k,2])
                    print('Added '+str(dT.iloc[k,2])+' tons of '+str(i)+' to port '+str(j)+'.')

            
                
inv.head()


Added 398.77 tons of MTA to port 564.
Added 21.93 tons of LSN_et_INC4 to port 564.
Added 487.51 tons of TMO to port 564.
Transferred 787.16 tons from 564 to 916.
Transferred 198.35 tons from 564 to 917.
Added 108.17 tons of CUZ to port 918.
Added 103.25 tons of STAL to port 918.
Added 286.57 tons of CLH to port 919.
Added 73.01 tons of BOLA to port 919.
Added 32.56 tons of BOLA to port 919.
Added 151.0 tons of NMP to port 920.
Added 26.47 tons of WOR to port 921.
Transferred 19.15 tons from 564 to 916.
Added 32.13 tons of MTA to port 564.
Added 993.23 tons of TMO to port 564.
Transferred 677.05 tons from 564 to 916.
Transferred 447.47 tons from 564 to 917.
Added 36.99 tons of CLH to port 918.
Added 125.35 tons of CUZ to port 918.
Added 108.2 tons of STAL to port 918.
Added 119.61 tons of CLH to port 919.
Added 36.97 tons of BOLA to port 919.
Added 27.7 tons of BOLA to port 919.
Added 23.58 tons of WOR to port 920.
Transferred 15.22 tons from 564 to 916.
Added 350.0 tons of LSN_et_INC4 

,564,560,900,903,906,913,915,916,917,918,919,920,921,922,923,924,925
XKM,98.158337,0.0,264.0,0.0,0.0,0.0,0.0,195.808096,73.033567,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
FLI,0.000000,0.0,604.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
MTA,122.392873,0.0,954.0,0.0,0.0,0.0,0.0,288.715385,85.791742,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Pyrite_don_rouyn_gallen,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
BCM,97.890876,0.0,178.0,0.0,0.0,0.0,0.0,195.274558,72.834565,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [20]:
# Check if any of the values are negative.
(inv.values<0).any()

False

In [21]:
inv

,564,560,900,903,906,913,915,916,917,918,919,920,921,922,923,924,925
XKM,98.158337,0.0,264.0,0.00,0.00,0.00,0.0,195.808096,73.033567,0.00,0.00,0.00,0.00,0.00,0.00,0.0,0.0
FLI,0.000000,0.0,604.0,0.00,0.00,0.00,0.0,0.000000,0.000000,0.00,0.00,0.00,0.00,0.00,0.00,0.0,0.0
MTA,122.392873,0.0,954.0,0.00,0.00,0.00,0.0,288.715385,85.791742,0.00,0.00,0.00,0.00,0.00,0.00,0.0,0.0
Pyrite_don_rouyn_gallen,0.000000,0.0,0.0,0.00,0.00,0.00,0.0,0.000000,0.000000,0.00,0.00,0.00,0.00,0.00,0.00,0.0,0.0
BCM,97.890876,0.0,178.0,0.00,0.00,0.00,0.0,195.274558,72.834565,0.00,0.00,0.00,0.00,0.00,0.00,0.0,0.0
DUM,0.000000,0.0,0.0,80.55,0.00,0.00,0.0,0.000000,0.000000,0.00,0.00,0.00,0.00,0.00,0.00,0.0,0.0
NVA,29.420755,0.0,0.0,44.00,0.00,0.00,0.0,68.689075,21.890170,0.00,0.00,0.00,0.00,0.00,0.00,0.0,0.0
LSN_et_INC4,5.865429,0.0,0.0,0.00,350.00,0.00,0.0,11.700467,4.364104,0.00,0.00,0.00,0.00,0.00,0.00,0.0,0.0
TIZ,0.000000,0.0,0.0,0.00,0.00,0.00,0.0,0.000000,0.000000,0.00,0.00,0.00,0.00,0.00,0.00,0.0,0.0
FAS,0.000000,0.0,0.0,0.00,0.00,0.00,0.0,0.000000,0.000000,0.00,0.00,0.00,0.00,0.00,0.00,0.0,0.0


In [22]:
# Read smelted transactions. This data is from the daily reactor report (rapport du reacteur)
# "Name" column for indication only. Matching uses "SourcePort" only.
smelt = pd.read_csv(r'C:\Users\win7\Documents\Fonte_data\smelt_2021-12-15.csv')
smelt.head()

,Name,SourcePort,DMT,Comment
0,Pyrite,498,1.9,NaN
1,Melange fil,916,1144.4,NaN
2,Vert conc,900,0.0,NaN
3,Tizapa,918,6.0,Added to Stall
4,CCR Residu,919,345.1,NaN


In [23]:
# Remove mass from inv according the smelt data
# Save all removed mass in a separate DataFrame
smelted = pd.Series(data=0, index=inv.index)
for row in range(len(smelt.SourcePort)):
    for inv_port in inv.columns:
        if str(smelt.loc[row,'SourcePort']) == inv_port:
            if smelt.loc[row,'DMT']>0:
                # Skip the inventory update for the port if it is empty.
                # Because it is impossible to know which material to assign negative mass in the port.
                if ~(inv.loc[:,inv_port]!=0).any(axis=0):
                    print(('Skipped removing inventory from port {} because it is empty.'
                          ' Cannot smelt from an empty port').format(inv_port))
                    continue
                # Calculate remaining fraction in the Eglise Port
                m = 1 - smelt.loc[row,'DMT'] / inv.loc[:,inv_port].sum()
                # Calculate removed mass by shipperID
                smelted += (1-m)*inv.loc[:,inv_port]
                # Update inventory
                inv.loc[:,inv_port] = inv.loc[:,inv_port] * m
                print('Removed {} dmt from port {}'.format(smelt.loc[row,'DMT'], inv_port))
                if m<0:
                    print('Caution, negative inventory in port {}: {} dmt.'.format(inv_port, inv.loc[:,inv_port].sum()))
                
                
inv.sum(axis=0)
        

Removed 1144.4 dmt from port 916
Removed 6.0 dmt from port 918
Removed 345.1 dmt from port 919
Removed 247.1 dmt from port 906
Removed 283.0 dmt from port 918
Removed 381.5 dmt from port 903
Caution, negative inventory in port 903: -256.95 dmt.
Removed 80.7 dmt from port 924
Removed 17.7 dmt from port 922
Skipped removing inventory from port 915 because it is empty. Cannot smelt from an empty port
Removed 120.3 dmt from port 920
Removed 349.9 dmt from port 913


564    1094.17
560     312.00
900    2000.00
903    -256.95
906     439.77
913     167.12
915       0.00
916     472.18
917     645.82
918     350.96
919     376.32
920     197.69
921     283.47
922      45.37
923     126.35
924      52.30
925     300.00
dtype: float64

Save smelted mass by shipperID

In [ ]:
smelted.to_csv('smelted'+date.today().strftime('%d-%m-%Y')+'.csv')

Show smelted materials

In [28]:
smelted

XKM                        138.615339
FLI                          0.000000
MTA                        204.385732
Pyrite_don_rouyn_gallen      0.000000
BCM                        138.237640
DUM                        246.726817
NVA                        183.399158
LSN_et_INC4                134.194672
TIZ                          0.000000
FAS                          0.000000
EAGL                       121.188255
NMP                        106.062408
STAL                       182.259235
DDG                          0.000000
FDN                         62.320248
CUZ                        138.202125
CLH                        278.608108
BOLA                        94.830507
HGC                          0.000000
Capela                      11.741359
TMO                        402.023295
UTE                          1.913448
ARUB                        74.443929
CEZ                          6.431334
CCR Tankhouse                0.000000
DOW                          0.000000
Coarse EOL  

In [45]:
# Group smelted categories
smeltCat = pd.read_csv(r'C:\Users\win7\Documents\Fonte_data\smeltCategories.csv')
smeltCat.head()

,Category,shipperCode
0,Conc Vert,XKM
1,Conc Vert,FLI
2,Conc Vert,MTA
3,Pyrite,Pyrite_don_rouyn_gallen
4,Conc Vert,BCM


In [46]:
# Check if shipperCodes match
(smeltCat['shipperCode']==smelted.index).all()

True

In [49]:
# Remove the null
smeltCat= smeltCat[smeltCat['Category'].notnull()]
smeltCat.head()

,Category,shipperCode
0,Conc Vert,XKM
1,Conc Vert,FLI
2,Conc Vert,MTA
3,Pyrite,Pyrite_don_rouyn_gallen
4,Conc Vert,BCM


In [59]:
# Make a dictionnary
smeltCat_dict = smeltCat.set_index('shipperCode')['Category'].to_dict()
smeltCat_dict
#shipID_dict = shipID.set_index('SourcePort')['ShipperID'].to_dict()

{'XKM': 'Conc Vert',
 'FLI': 'Conc Vert',
 'MTA': 'Conc Vert',
 'Pyrite_don_rouyn_gallen': 'Pyrite',
 'BCM': 'Conc Vert',
 'DUM': 'Conc Vert',
 'NVA': 'NVA',
 'LSN_et_INC4': 'Conc Ni',
 'TIZ': 'TIZ',
 'FAS': 'Conc Ni',
 'EAGL': 'Conc Ni',
 'STAL': 'STAL',
 'DDG': 'DDG',
 'FDN': 'FDN',
 'CUZ': 'CUZ',
 'CLH': 'CLH',
 'BOLA': 'BOLA',
 'HGC': 'HGC',
 'Capela': 'Capela',
 'TMO': 'TMO'}

In [64]:
smelted = pd.DataFrame(smelted, columns=['DMT'])

In [75]:
# Map categories to shipperCodes in smelted
#dT['shipperID'] = dT['sourcePort'].map(shipID_dict)
smelted['Category'] = smelted.index.map(smeltCat_dict)
smelted.groupby('Category').sum().sort_values('DMT', ascending=False)

,DMT
Category,
Conc Vert,727.965528
TMO,402.023295
CLH,278.608108
Conc Ni,255.382927
NVA,183.399158
STAL,182.259235
CUZ,138.202125
BOLA,94.830507
FDN,62.320248


In [25]:

smelted.sum()

2975.7

In [ ]:
smelt['DMT'].sum()

In [ ]:
'smelted'+date.today().strftime('%d-%m-%Y')+'.csv'